In [28]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, recall_score

from keras.models import load_model

from settings import *
import eb_models
from preprocessing import load_npz, make_dataset, normalize

In [2]:
cut_len = 1024

In [36]:
h5list = sorted(glob.glob(os.path.join(modeldir, "16*5_?.h5")))

In [37]:
data0_list = []
data1_list = []
for sector in [1, 2, 3, 5]:
    npzpath0 = os.path.join(datdir, "%s_0.npz" % sector)
    npzpath1 = os.path.join(datdir, "%s_1.npz" % sector)
    data1 = load_npz(npzpath1)
#     data0 = load_npz(npzpath0)[:data1.shape[0]]
    data0 = load_npz(npzpath0)[:]
    data0 = make_dataset(data0, cut_len)
    data1 = make_dataset(data1, cut_len)
    data0_list.append(data0)
    data1_list.append(data1)
data0 = np.vstack(tuple(data0_list))
data1 = np.vstack(tuple(data1_list))
label0 = np.zeros(data0.shape[0])
label1 = np.ones(data1.shape[0])
data = np.vstack((data0, data1))
label = np.hstack((label0, label1))
data = normalize(data)

In [38]:
data = np.reshape(data, (data.shape[0], data.shape[1], 1))
label = np.reshape(label, (label.shape[0], 1))
data1 = np.reshape(data1, (data1.shape[0], data1.shape[1], 1))
label1 = np.reshape(label1, (label1.shape[0], 1))

In [39]:
# , custom_objects={"class_name": class_name}
for h5path in h5list:
    model = load_model(h5path, compile=True)
#     score = model.evaluate(data, label, verbose=0)
    pred = np.round(model.predict(data))
    tp, fn, fp, tn = confusion_matrix(label, pred).ravel()
    print(tp, fn, fp, tn, os.path.basename(h5path))

115894 2498 1616 10292 16_16_32_5_0.h5
116052 2340 1480 10428 16_16_32_5_1.h5
114288 4104 1112 10796 16_16_32_5_2.h5
115674 2718 708 11200 16_16_32_5_3.h5
113084 5308 1056 10852 16_16_64_5_0.h5
114900 3492 770 11138 16_16_64_5_1.h5
114880 3512 1030 10878 16_16_64_5_2.h5
114266 4126 610 11298 16_16_64_5_3.h5
113240 5152 1156 10752 16_32_32_5_0.h5
116786 1606 1016 10892 16_32_32_5_1.h5
113742 4650 1000 10908 16_32_32_5_2.h5
114746 3646 666 11242 16_32_32_5_3.h5
113662 4730 950 10958 16_32_64_5_0.h5
113180 5212 624 11284 16_32_64_5_1.h5
111726 6666 848 11060 16_32_64_5_2.h5
115440 2952 784 11124 16_32_64_5_3.h5
